<a href="https://colab.research.google.com/github/vsalavatov/pytorch_resnet_cifar10/blob/master/ResNet_20_Consensus_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! apt-get install python3.7 python3.7-dev gcc
! curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
! python3.7 get-pip.py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
gcc set to manually installed.
python3.7 is already the newest version (3.7.10-1+bionic2).
python3.7-dev is already the newest version (3.7.10-1+bionic2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1882k  100 1882k    0     0  4902k      0 --:--:-- --:--:-- --:--:-- 4902k
     |████████████████████████████████| 1.5 MB 6.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [20]:
%cd /content
! rm -rf pytorch_resnet_cifar10

/content


In [21]:
! git clone https://github.com/vsalavatov/pytorch_resnet_cifar10
%cd pytorch_resnet_cifar10

Cloning into 'pytorch_resnet_cifar10'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 204 (delta 18), reused 27 (delta 12), pack-reused 168
Receiving objects: 100% (204/204), 84.47 MiB | 37.51 MiB/s, done.
Resolving deltas: 100% (104/104), done.
/content/pytorch_resnet_cifar10


In [4]:
! pip3 install -r requirements.txt

In [5]:
import os
os.environ['CHECKPOINT_PATH'] = './data'
! echo $CHECKPOINT_PATH

./data


In [22]:
! cd distributed-learning && git submodule update --recursive --init
! echo '' > distributed-learning/utils/__init__.py

Submodule 'distributed-learning' (https://github.com/vsalavatov/distributed-learning.git) registered for path './'
Cloning into '/content/pytorch_resnet_cifar10/distributed-learning'...
Submodule path './': checked out 'ec6fb67fbcae296cd51ee043b34c537a8ce65f73'
Submodule 'wide-resnet.pytorch' (https://github.com/meliketoy/wide-resnet.pytorch.git) registered for path './wide_resnet_submodule'
Cloning into '/content/pytorch_resnet_cifar10/distributed-learning/wide_resnet_submodule'...
Submodule path './wide_resnet_submodule': checked out '292b3ede0651e349dd566f9c23408aa572f1bd92'


In [7]:
! cd distributed-learning && pip3 install -r requirements.txt

     |████████████████████████████████| 132 kB 6.5 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219292 sha256=d36c80ede4ac9baf1bac05ef547b5d539756e751dcbce23ceb86c59cb0a28009
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Successfully built pickle5
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3.7 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-3t_xvvpy/pytorch_898cf38769284b4084d8e94739f65ee6/setup.py'"'"'; __file__='"'"'/tmp/pip-install-3t_xvvpy/pytorch_898cf38769284b4084d8e94739f65ee6/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip

In [8]:
! pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8 MB 22 kB/s 
     |████████████████████████████████| 6.6 MB 8.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cu101
    Uninstalling torch-1.7.1+cu101:
      Successfully uninstalled torch-1.7.1+cu101
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2+cu101
    Uninstalling torchvision-0.8.2+cu101:
      Successfully uninstalled torchvision-0.8.2+cu101


In [9]:
! pip install cvxpy

# If you want to connect GDrive to store checkpoints, run this

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# DO NOT FORGET TO MODIFY THE PATH
import os
os.environ['CHECKPOINT_PATH'] = '/content/gdrive/MyDrive/' + 'resnet20-plain-consensus-batch32-k5-freq=1/'
! echo $CHECKPOINT_PATH

/content/gdrive/MyDrive/resnet20-lsr-consensus-batch32-k5-round=e-4/


# Training stage


In [23]:
!nvidia-smi

Fri Mar  5 18:39:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
%cd /content/pytorch_resnet_cifar10/

/content/pytorch_resnet_cifar10


In [25]:
! python3.7 prepare_agent_datasets.py -n 5

170500096it [00:02, 71103530.67it/s]                    
Extracting ./data/cifar-10-python.tar.gz to ./data
10000 samples for agent 0
10000 samples for agent 1
10000 samples for agent 2
10000 samples for agent 3
10000 samples for agent 4


In [26]:
!pkill -9 -f python3\.7.*
%cd /content/pytorch_resnet_cifar10/

/content/pytorch_resnet_cifar10


In [ ]:
# check out "run_consensus.py --help" to see customizable options

! python3.7 run_consensus.py -n 5 --topology mesh --validation-agents 0,3 

In [ ]:
! ps -aux